In [236]:
import re

In [237]:
from astropy.io import ascii
from astroquery.heasarc import Heasarc
from astroquery.vizier import Vizier
from astroquery.ipac.ned import Ned
import pandas as pd
import numpy as np

## Query Ricci+17

In [ ]:
v = Vizier(catalog="J/ApJS/233/17/Xraydata",columns=['SWIFT','logNH','b_logNH','B_logNH','Model'],row_limit=-1)
# http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=J/ApJS/80/531/gxfluxes
print(v.columns)
data = v.query_constraints()

## Query Koss+23

In [239]:
v = Vizier(catalog="J/ApJS/261/2/table9",columns=['ID','SWIFT','CName','RAJ2000','DEJ2000','type','z','ztype'],row_limit=-1)
# http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=J/ApJS/80/531/gxfluxes
print(v.columns)
data = v.query_constraints()


['ID', 'SWIFT', 'CName', 'RAJ2000', 'DEJ2000', 'type', 'z', 'ztype']


In [240]:
ascii.write(data[0],"test.txt",delimiter='|',overwrite=True)

In [241]:
df = pd.read_table('test.txt', header=0,delimiter="|")

In [242]:
df["SWIFT"] = "SWIFT" + df["SWIFT"]

In [ ]:
df.to_csv('name.csv', header=False, index=False, columns=['SWIFT'])

In [ ]:
df.to_csv('bassradec.csv', header=False, index=False, columns=['RAJ2000','DEJ2000'])

In [243]:
df

,ID,SWIFT,CName,RAJ2000,DEJ2000,z,ztype
0,1,SWIFTJ0001.0-0708,2MASXJ00004876-0709117,0.203234,-7.153223,0.037496,OIII
1,2,SWIFTJ0001.6-7701,2MASXJ00014596-7657144,0.442035,-76.953963,0.058505,OIII Broad
2,3,SWIFTJ0002.5+0323,NGC7811,0.610105,3.351912,0.025457,OIII Broad
3,4,SWIFTJ0003.3+2737,2MASXJ00032742+2739173,0.864248,27.654725,0.039784,OIII
4,5,SWIFTJ0005.0+7021,2MASXJ00040192+7019185,1.008241,70.321752,0.095681,OIII
...,...,...,...,...,...,...,...
853,1209,SWIFTJ2359.0-3038,H2356-309,359.782919,-30.627960,0.165138,Gal Temp
854,1210,SWIFTJ2359.3-6058,PKS2356-61,359.767938,-60.916440,0.096211,OIII
855,1077,SWIFTJ2028.5+2543,MCG+04-48-002,307.146087,25.733383,0.013272,OIII
856,112,SWIFTJ0209.5-1010,NGC833,32.336859,-10.133086,0.013548,OIII


In [ ]:
df[df["SWIFT"]=="SWIFTJ0456.3-7532"]

## Query HEASARC

In [ ]:
df = pd.read_table('basstable.csv', header=0,delimiter=",")
df["Counterpart"].iloc[0]

In [ ]:
heasarc = Heasarc()

In [ ]:
heasarc.query_mission_cols(mission='numaster')

In [ ]:
heasarc.query_mission_cols(mission='xmmmaster')

In [ ]:
heasarc.query_mission_cols(mission='suzamaster')

In [ ]:
table1 = heasarc.query_object("2MASXJ03352254+1907282",mission='numaster', fields='name,ra,dec,time,obsid,status,exposure_a,observation_mode,public_date,issue_flag,end_time')
#table1["NAME"] = df["SwiftName"].iloc[0]
table1


In [ ]:
table = heasarc.query_object(df["Counterpart"].iloc[0],mission='numaster', fields='name,ra,dec,time,obsid,status,exposure_a,observation_mode,public_date,issue_flag,end_time')
table["NAME"] = df["SwiftName"].iloc[0]
table

In [ ]:
for i in range(1,len(df)):
    t = heasarc.query_object(df["Counterpart"].iloc[i],mission='numaster', fields='name,ra,dec,time,obsid,status,exposure_a,observation_mode,public_date,issue_flag,end_time')
    t["NAME"] = df["SwiftName"].iloc[i]
    for arr in np.array(t):
        table.add_row(arr)

In [ ]:
table

In [ ]:
np.array(table)

In [ ]:
table.add_row(np.array(table)[0])

In [ ]:
table

In [ ]:
nustar_df = pd.read_table('nustarbass.txt', header=0,delimiter="|")
suzaku_df = pd.read_csv('suzakubass.txt', header=0,delimiter="|")

In [ ]:
pd.to_datetime(nustar_df[nustar_df["status"]=="archived "]['time'])

In [ ]:
nustar_df.columns = nustar_df.columns.str.strip()
suzaku_df.columns = suzaku_df.columns.str.strip()

In [ ]:
nustar_df

## Query NED

In [ ]:
result_table = Ned.query_object("SWIFTJ0456.3-7532")

In [ ]:
result_table

## NhTot

In [244]:
dfnh = pd.read_table('bassnhtot.txt', header=0,delimiter="|")

In [245]:
dfnh

,Position,E(B-V)_Mean,E(B-V)_Weighted,NHI_Mean,NHI_Weighted,NH2_Mean,NH2_Weighted,NHtot_Mean,NHtot_Weighted,Unnamed: 9
0,"J 0.20323, -7.15322",0.036,0.037,3.150000e+20,3.130000e+20,3.870000e+19,3.970000e+19,3.540000e+20,3.520000e+20,NaN
1,"J 0.44203, -76.95396",0.053,0.055,4.650000e+20,4.810000e+20,8.770000e+19,9.470000e+19,5.530000e+20,5.760000e+20,NaN
2,"J 0.61011, 3.35191",0.023,0.023,2.810000e+20,2.810000e+20,2.090000e+19,2.040000e+19,3.010000e+20,3.010000e+20,NaN
3,"J 0.86425, 27.65473",0.052,0.055,4.250000e+20,4.300000e+20,7.880000e+19,8.500000e+19,5.030000e+20,5.150000e+20,NaN
4,"J 1.00824, 70.32175",0.970,1.033,3.340000e+21,3.480000e+21,1.440000e+21,1.440000e+21,4.780000e+21,4.920000e+21,NaN
...,...,...,...,...,...,...,...,...,...,...
853,"J 359.78292, -30.62796",0.015,0.015,1.420000e+20,1.440000e+20,6.170000e+18,6.250000e+18,1.480000e+20,1.500000e+20,NaN
854,"J 359.76794, -60.91644",0.015,0.013,1.560000e+20,1.510000e+20,6.720000e+18,5.760000e+18,1.630000e+20,1.570000e+20,NaN
855,"J 307.14609, 25.73338",0.479,0.455,2.150000e+21,2.070000e+21,1.390000e+21,1.370000e+21,3.540000e+21,3.450000e+21,NaN
856,"J 32.33686, -10.13309",0.024,0.024,2.500000e+20,2.530000e+20,1.930000e+19,1.990000e+19,2.690000e+20,2.730000e+20,NaN


In [246]:
df['NHtot_Weighted'] = dfnh['NHtot_Weighted']

In [247]:
df

,ID,SWIFT,CName,RAJ2000,DEJ2000,z,ztype,NHtot_Weighted
0,1,SWIFTJ0001.0-0708,2MASXJ00004876-0709117,0.203234,-7.153223,0.037496,OIII,3.520000e+20
1,2,SWIFTJ0001.6-7701,2MASXJ00014596-7657144,0.442035,-76.953963,0.058505,OIII Broad,5.760000e+20
2,3,SWIFTJ0002.5+0323,NGC7811,0.610105,3.351912,0.025457,OIII Broad,3.010000e+20
3,4,SWIFTJ0003.3+2737,2MASXJ00032742+2739173,0.864248,27.654725,0.039784,OIII,5.150000e+20
4,5,SWIFTJ0005.0+7021,2MASXJ00040192+7019185,1.008241,70.321752,0.095681,OIII,4.920000e+21
...,...,...,...,...,...,...,...,...
853,1209,SWIFTJ2359.0-3038,H2356-309,359.782919,-30.627960,0.165138,Gal Temp,1.500000e+20
854,1210,SWIFTJ2359.3-6058,PKS2356-61,359.767938,-60.916440,0.096211,OIII,1.570000e+20
855,1077,SWIFTJ2028.5+2543,MCG+04-48-002,307.146087,25.733383,0.013272,OIII,3.450000e+21
856,112,SWIFTJ0209.5-1010,NGC833,32.336859,-10.133086,0.013548,OIII,2.730000e+20


In [248]:
df.to_csv('basstable.csv', header=['ID','SwiftName','Counterpart','RA','DEC','Redshift','nh'], index=False, columns=['ID','SWIFT','CName','RAJ2000','DEJ2000','z','NHtot_Weighted'])

In [249]:
df = pd.read_table('basstable.csv', header=0,delimiter=",")

In [250]:
df

,ID,SwiftName,Counterpart,RA,DEC,Redshift,nh
0,1,SWIFTJ0001.0-0708,2MASXJ00004876-0709117,0.203234,-7.153223,0.037496,3.520000e+20
1,2,SWIFTJ0001.6-7701,2MASXJ00014596-7657144,0.442035,-76.953963,0.058505,5.760000e+20
2,3,SWIFTJ0002.5+0323,NGC7811,0.610105,3.351912,0.025457,3.010000e+20
3,4,SWIFTJ0003.3+2737,2MASXJ00032742+2739173,0.864248,27.654725,0.039784,5.150000e+20
4,5,SWIFTJ0005.0+7021,2MASXJ00040192+7019185,1.008241,70.321752,0.095681,4.920000e+21
...,...,...,...,...,...,...,...
853,1209,SWIFTJ2359.0-3038,H2356-309,359.782919,-30.627960,0.165138,1.500000e+20
854,1210,SWIFTJ2359.3-6058,PKS2356-61,359.767938,-60.916440,0.096211,1.570000e+20
855,1077,SWIFTJ2028.5+2543,MCG+04-48-002,307.146087,25.733383,0.013272,3.450000e+21
856,112,SWIFTJ0209.5-1010,NGC833,32.336859,-10.133086,0.013548,2.730000e+20


In [ ]:
df1 = df[df["SwiftName"] == "SWIFTJ2359.3-6058"]

In [ ]:
df1["RA"].values[0]

In [ ]:
int(ra.values/15)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_table('basstable.csv', header=0,delimiter=",",dtype=str)

In [ ]:
nustar_df = pd.read_table('nustarbass.txt', header=0,delimiter="|")
nustar_df.columns = nustar_df.columns.str.strip()
for i in range(len(nustar_df)):
    ra=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',nustar_df['_Search_Offset'].iloc[i]).group(1)
    dec=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',nustar_df['_Search_Offset'].iloc[i]).group(2)

    #df[(df["RA"]==ra) & (dec in df["DEC"])]['SwiftName'].iloc[0]
    nustar_df['SwiftName'].iloc[i] = df[(df["RA"]==ra)]['SwiftName'].iloc[0]

In [ ]:
#nustar_df=nustar_df.drop(columns=['Unnamed: 0', 'Unnamed: 13'])
nustar_df=nustar_df.reindex(columns=['SwiftName',*nustar_df.columns[:-1]])
nustar_df.to_csv('nustarbass.csv', index=False)

In [ ]:
suzaku_df = pd.read_table('suzakubass.txt', header=0,delimiter="|")
suzaku_df.columns = suzaku_df.columns.str.strip()
suzaku_df['SwiftName'] = ""
for i in range(len(suzaku_df)):
    ra=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',suzaku_df['_Search_Offset'].iloc[i]).group(1)
    dec=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',suzaku_df['_Search_Offset'].iloc[i]).group(2)

    #df[(df["RA"]==ra) & (dec in df["DEC"])]['SwiftName'].iloc[0]
    suzaku_df['SwiftName'].iloc[i] = df[(df["RA"]==ra)]['SwiftName'].iloc[0]
suzaku_df

In [ ]:
suzaku_df=suzaku_df.drop(columns=['Unnamed: 0', 'Unnamed: 10'])
suzaku_df=suzaku_df.reindex(columns=['SwiftName',*suzaku_df.columns[:-1]])
suzaku_df.to_csv('suzakubass.csv', index=False)

In [ ]:
xmm_df = pd.read_table('xmmbass.txt', header=0,delimiter="|",dtype=str)
xmm_df.columns = xmm_df.columns.str.strip()
xmm_df['SwiftName'] = ""
for i in range(len(xmm_df)):
    ra=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',xmm_df['_Search_Offset'].iloc[i]).group(1)
    dec=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',xmm_df['_Search_Offset'].iloc[i]).group(2)

    #df[(df["RA"]==ra) & (dec in df["DEC"])]['SwiftName'].iloc[0]
    xmm_df['SwiftName'].iloc[i] = df[(df["RA"]==ra)]['SwiftName'].iloc[0]
xmm_df

In [ ]:
xmm_df=xmm_df.drop(columns=['Unnamed: 0', 'Unnamed: 12'])
xmm_df=xmm_df.reindex(columns=['SwiftName',*xmm_df.columns[:-1]])
xmm_df.to_csv('xmmbass.csv', index=False)

In [ ]:
chandra_df = pd.read_table('chandrabass.txt', header=0,delimiter="|")
chandra_df.columns = chandra_df.columns.str.strip()
chandra_df['SwiftName'] = ""
for i in range(len(xmm_df)):
    ra=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',chandra_df['_Search_Offset'].iloc[i]).group(1)
    dec=re.search(r'\((\d+\.\d+),(-?\d+\.\d+).+\)',chandra_df['_Search_Offset'].iloc[i]).group(2)

    #df[(df["RA"]==ra) & (dec in df["DEC"])]['SwiftName'].iloc[0]
    chandra_df['SwiftName'].iloc[i] = df[(df["RA"]==ra)]['SwiftName'].iloc[0]
chandra_df

In [ ]:
chandra_df=chandra_df.drop(columns=['Unnamed: 0', 'Unnamed: 12'])
chandra_df=chandra_df.reindex(columns=['SwiftName',*chandra_df.columns[:-1]])
chandra_df.to_csv('chandrabass.csv', index=False)

In [ ]:

nustar_df = pd.read_table('nustarbass.csv', header=0,delimiter=",",parse_dates=['time'])
nustar_df

In [ ]:
nustar_df.dtypes

## For Spitzer https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-dd?catalog=irs_enhv211

In [254]:
with open('bassradec.csv','r') as f:
    bassradec = f.readlines()
bassradec = [i.strip('\n').split(',') for i in bassradec]

In [256]:
bassradec = [' {0:15} {1:15}'.format(*i) for i in bassradec]

In [258]:
header = ["\ EQUINOX = 'J2000.0'",
'|{0:>15}|{1:15}|'.format('ra','dec'),
'|{0:>15}|{1:15}|'.format('double','double')]

In [260]:
with open('bassradec_spitzer.csv','w') as f:
    f.write('\n'.join(header+bassradec))